In [2]:
import pandas as pd
import numpy as np
import os
import json
from PIL import Image
from io import StringIO

In [ ]:
import os
import google.generativeai as genai

# Set the GOOGLE_API_KEY in the environment first

# Then configure the genai module with the API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Now you can create the GenerativeModel instance
model = genai.GenerativeModel('gemini-2.0-flash-001')

In [4]:
!wget https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar

--2025-05-12 05:38:13--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 54.231.130.249, 54.231.130.145, 16.15.177.135, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|54.231.130.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3253381120 (3.0G) [application/x-tar]
Saving to: ‘abo-images-small.tar’

abo-images-small.ta 100%[===================>]   3.03G  49.1MB/s    in 75s     

2025-05-12 05:39:28 (41.4 MB/s) - ‘abo-images-small.tar’ saved [3253381120/3253381120]



In [5]:
!tar -xf abo-images-small.tar

In [6]:
listing_file = "/content/drive/MyDrive/abo-listings/listings/metadata/listings_2.json.gz"
image_metadata = pd.read_csv("/content/drive/MyDrive/images/metadata/images.csv")
image_dataset_path = "/content/images/small"

In [7]:
#create output csv file
output_file = f"/content/drive/MyDrive/images/{listing_file.split('/')[-1].split('.')[0]}_VQA.csv"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# def get_listing_lines(listing_file):
#     """
#     Read the listing file and return a list of lines.
#     """
#     with open(listing_file, 'r') as file:
#         lines = file.readlines()
#     return lines
import gzip

def get_listing_lines(listing_file):
    """
    Read the listing file and return a list of lines.
    Handles gzip compression if the file name ends with '.gz'.
    """
    if listing_file.endswith('.gz'):
        with gzip.open(listing_file, 'rt', encoding='utf-8') as file:
            lines = file.readlines()
    else:
        with open(listing_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    return lines


In [9]:
def preprocess_product_json(product_json):
        """
        Preprocess the product JSON to ensure it is in the correct format.
        """
        list_of_keys_to_remove = ['main_image_id','node','other_image_id','spin_id','3dmodel_id']
        # Convert JSON string to dictionary
        product_dict = json.loads(product_json)
        # Remove unnecessary keys
        for key in list_of_keys_to_remove:
            if key in product_dict:
                del product_dict[key]

        # Convert dictionary to JSON string with indentation for better readability
        return json.dumps(product_dict, indent=4)
def prompt_for_product(product_json):
        """
        Generate a prompt for the given product JSON.
        """
        preprocess_product_json(product_json)
        prompt = f"""
        You are a QA dataset generator that creates short, factual, and human-readable question-answer pairs from Amazon product metadata and image. Each question must target a specific field from the metadata and be answerable with a *single word only*.

        Below is the product metadata in structured format. Generate *5 to 10 diverse QA pairs*, where:
        - Each question is clear and unambiguous.
        - Each answer is strictly a *single word* (no phrases, no multi-word answers).
        - Avoid repeating the same field.
        - Prefer commonly relevant fields like: brand, bullet_points, color, material, product type, model name, style, fabric type, finish type, pattern, item shape, product description and color code.
        - Questions should be such a way that they can be answered by looking at the image.
        - The output should be in CSV format with columns: question, answer.

        If a value is not meaningful or not present, skip that field. Ensure that QA pairs are diverse and aligned with the data provided.

        ---
        {product_json}
        ---
        """

        return prompt

In [10]:
def get_images_paths(image_ids):
    """
    Get the paths of images based on their IDs.
    """
    image_paths = []
    for image_id in image_ids:
        image_path = image_metadata[image_metadata['image_id'] == image_id]['path'].values
        if len(image_path) > 0:
            if os.path.exists(f"{image_dataset_path}/{image_path[0]}"):
                image_paths.append(f"{image_path[0]}")
    return image_paths

def generate_VQA(prompt, image_path):
    img = Image.open(f"{image_dataset_path}/{image_path}")
    img = img.convert("RGB")
    # Generate the VQA using the model
    response = model.generate_content([prompt, img])
    # Extract the generated text from the response
    generated_text = response.text
    #read csv from the generated text
    csv_data = pd.read_csv(StringIO(generated_text.strip("`").replace("csv\n", "", 1).strip()))
    return csv_data

def get_VQA_for_product(product_json):
    df = pd.DataFrame(columns=["image_path","question", "answer"])
    list_of_image_ids = []

    prompt = prompt_for_product(product_json)
    product_dict = json.loads(product_json)
    list_of_image_ids.append(product_dict['main_image_id'])
    if "other_image_id" in product_dict.keys():
        # Check if the key exists in the dictionary
        if isinstance(product_dict['other_image_id'], list):
            # If it's a list, extend it to the list_of_image_ids
            list_of_image_ids.extend(product_dict['other_image_id'])
        else:
            # If it's not a list, append it directly
            list_of_image_ids.append(product_dict['other_image_id'])
    image_paths = get_images_paths(list_of_image_ids)

    print(f"Image paths: {image_paths}")
    # Generate the VQA using the model
    for image_path in image_paths:
        print(f"Generating VQA for image: {image_path}")
        # Generate the VQA using the model
        csv_data = generate_VQA(prompt, image_path)
        # Append the generated data to the dataframe
        csv_data['image_path'] = image_path
        df = pd.concat([df, csv_data], ignore_index=True)
    return df


In [12]:
from tqdm import tqdm

# Prepare existing CSV
if not os.path.exists(output_file):
    output_df = pd.DataFrame(columns=["image_path", "question", "answer"])
else:
    output_df = pd.read_csv(output_file)

# Track processed image_ids
processed_ids = set(output_df['image_path'].unique())

# Load metadata lines
lines = get_listing_lines(listing_file)
subset_lines = lines[1100:]

try:
    for line in tqdm(subset_lines):
        if "\"en_" not in line:
            continue

        try:
            product_dict = json.loads(line)
            main_image_id = product_dict.get('main_image_id', None)
            if not main_image_id or main_image_id in processed_ids:
                continue
        except Exception:
            continue

        df = get_VQA_for_product(line)
        output_df = pd.concat([output_df, df], ignore_index=True)

        # Update processed_ids dynamically to reduce memory usage
        processed_ids.update(df['image_path'].unique())

        # Save incrementally to avoid data loss
        output_df.to_csv(output_file, index=False)

except Exception as e:
    print(f"An error occurred: {e}")
    pass


  0%|          | 0/8132 [00:00<?, ?it/s]

Image paths: ['3e/3e7947a9.jpg', 'a7/a7f639f5.jpg', '26/26f8865d.jpg', '1c/1cc7c5cb.jpg', '18/1869d79b.jpg', 'f1/f1109cdc.jpg', 'af/af42b343.jpg', '48/48069583.jpg', 'ad/adbac514.jpg', '7f/7f6e79b7.jpg', 'ea/ea290b15.jpg', 'c2/c2eda3e8.jpg']
Generating VQA for image: 3e/3e7947a9.jpg
Generating VQA for image: a7/a7f639f5.jpg
Generating VQA for image: 26/26f8865d.jpg
Generating VQA for image: 1c/1cc7c5cb.jpg
Generating VQA for image: 18/1869d79b.jpg
Generating VQA for image: f1/f1109cdc.jpg
Generating VQA for image: af/af42b343.jpg
Generating VQA for image: 48/48069583.jpg
Generating VQA for image: ad/adbac514.jpg
Generating VQA for image: 7f/7f6e79b7.jpg
Generating VQA for image: ea/ea290b15.jpg
Generating VQA for image: c2/c2eda3e8.jpg


  0%|          | 1/8132 [00:28<64:41:48, 28.64s/it]

Image paths: ['72/72b0c083.jpg', 'ee/eeed7f53.jpg', 'aa/aa0c1f43.jpg', 'bf/bf2fb6b5.jpg', '25/25637dd0.jpg']
Generating VQA for image: 72/72b0c083.jpg
Generating VQA for image: ee/eeed7f53.jpg
Generating VQA for image: aa/aa0c1f43.jpg
Generating VQA for image: bf/bf2fb6b5.jpg
Generating VQA for image: 25/25637dd0.jpg


  0%|          | 2/8132 [00:37<38:32:49, 17.07s/it]

...

Image paths: ['e7/e761e813.jpg', '52/528f18c3.jpg', 'f3/f31c1419.jpg', '8d/8d63483d.jpg', '82/82faf804.jpg', 'fd/fdc6a680.jpg', '8b/8b645378.jpg', '7f/7f1b8930.jpg']
Generating VQA for image: e7/e761e813.jpg
Generating VQA for image: 52/528f18c3.jpg
Generating VQA for image: f3/f31c1419.jpg
Generating VQA for image: 8d/8d63483d.jpg
Generating VQA for image: 82/82faf804.jpg
Generating VQA for image: fd/fdc6a680.jpg
Generating VQA for image: 8b/8b645378.jpg
Generating VQA for image: 7f/7f1b8930.jpg


  6%|▌         | 496/8132 [1:01:52<23:28:07, 11.06s/it]

Image paths: ['77/77d6bd98.jpg', '69/6906a09b.jpg', '87/87f287c6.jpg', '26/26e15418.jpg', '40/4082582d.jpg', 'c6/c678fb25.jpg', '08/0808e956.jpg', 'f0/f08321bf.jpg']
Generating VQA for image: 77/77d6bd98.jpg


  6%|▌         | 496/8132 [1:01:54<15:52:58,  7.49s/it]

An error occurred: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
